In [1]:
import numpy as np
import ray
import time

print('Successfully imported ray!')

Successfully imported ray!


In [2]:
ray.init(num_cpus=6, ignore_reinit_error=True)

# Sleep a little to improve the accuracy of the timing measurements used below,
# because some workers may still be starting up in the background.
time.sleep(2.0)

2019-12-20 17:41:34,116	WARNING services.py:597 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2019-12-20 17:41:34,122	INFO resource_spec.py:216 -- Starting Ray with 1.76 GiB memory available for workers and up to 0.88 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


# 1. Handling Slow Tasks

In [3]:
@ray.remote
def f(i):
    np.random.seed(5 + i)
    x = np.random.uniform(0, 4)
    time.sleep(x)
    return i, time.time()

In [4]:
start_time = time.time()

# This launches 6 tasks, each of which takes a random amount of time to
# complete.
result_ids = [f.remote(i) for i in range(6)]
# Get one batch of tasks. Instead of waiting for a fixed subset of tasks, we
# should instead use the first 3 tasks that finish.
ready_ids, remaining_ids = ray.wait(result_ids, 3)
initial_results = ray.get(ready_ids)

end_time = time.time()
duration = end_time - start_time

In [5]:
# Wait for the remaining tasks to complete.
remaining_results = ray.get(remaining_ids)

In [6]:
assert len(initial_results) == 3
assert len(remaining_results) == 3

initial_indices = [result[0] for result in initial_results]
initial_times = [result[1] for result in initial_results]
remaining_indices = [result[0] for result in remaining_results]
remaining_times = [result[1] for result in remaining_results]

assert set(initial_indices + remaining_indices) == set(range(6))

assert duration < 1.5, ('The initial batch of ten tasks was retrieved in '
                        '{} seconds. This is too slow.'.format(duration))

assert duration > 0.8, ('The initial batch of ten tasks was retrieved in '
                        '{} seconds. This is too slow.'.format(duration))

# Make sure the initial results actually completed first.
assert max(initial_times) < min(remaining_times)

print('Success! The example took {} seconds.'.format(duration))

Success! The example took 0.9022083282470703 seconds.


# 2. Process Tasks in Order of Completion 
We have a list of unfinished tasks and recursively call `ray.wait()` and update unfinished tasks as there is a
change. The ready ones are processed as necessary

In [7]:
ray.init(num_cpus=5, include_webui=False, ignore_reinit_error=True)

2019-12-20 18:10:09,203	ERROR worker.py:679 -- Calling ray.init() again after it has already been called.


In [8]:
@ray.remote
def f():
    time.sleep(np.random.uniform(0, 5))
    return time.time()

In [9]:
start_time = time.time()

remaining_result_ids = [f.remote() for _ in range(10)]

# Get the results.
results = []
while len(remaining_result_ids) > 0:
    # EXERCISE: Instead of simply waiting for the first result from
    # remaining_result_ids, use ray.wait to get the first one to finish.
    result_id, remaining_result_ids = ray.wait(remaining_result_ids)

    result = ray.get(result_id[0])
    results.append(result)

    print('Processing result which finished after {} seconds.'
          .format(result - start_time))    

end_time = time.time()
duration = end_time - start_time

Processing result which finished after 0.1294393539428711 seconds.
Processing result which finished after 1.674577236175537 seconds.
Processing result which finished after 2.539151191711426 seconds.
Processing result which finished after 3.3020849227905273 seconds.
Processing result which finished after 3.9239449501037598 seconds.
Processing result which finished after 4.381956338882446 seconds.
Processing result which finished after 4.871961832046509 seconds.
Processing result which finished after 5.02244758605957 seconds.
Processing result which finished after 5.785333871841431 seconds.
Processing result which finished after 7.051407098770142 seconds.


In [10]:
assert results == sorted(results), ('The results were not processed in the '
                                    'order that they finished.')

print('Success! The example took {} seconds.'.format(duration))


Success! The example took 7.052814483642578 seconds.
